Import Library yang dibutuhkan

In [3]:
import numpy as np
import nltk
nltk.download('punkt_tab')
from nltk.stem import WordNetLemmatizer
import json
import pickle
import random

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation, Dropout
from tensorflow.keras.optimizers import SGD

from keras.models import load_model

[nltk_data] Downloading package punkt_tab to C:\Users\WORK
[nltk_data]     PLUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [4]:
import nltk
print(nltk.data.find('tokenizers/punkt'))


C:\Users\WORK PLUS\AppData\Roaming\nltk_data\tokenizers\punkt


In [5]:
import os
print(os.path.exists('C:/Users/WORK PLUS/AppData/Roaming/nltk_data/tokenizers/punkt'))


True


In [6]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

words=[]
classes=[]
documents=[]
ignore=['?','!',',',"'s"]


In [7]:
dataset = open(r'intents.json').read()
intents = json.loads(dataset)

In [8]:
print(intents)

{'intents': [{'tag': 'google', 'patterns': ['google', 'search', 'internet'], 'responses': ['Redirecting to Google...']}, {'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'halo', 'Hello', 'Good day', 'Namaste', 'yo'], 'responses': ['Hello', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Get lost', 'Till next time', 'bbye'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you

In [9]:
nltk.data.path.append('C:\\Users\\WORK PLUS\\AppData\\Roaming\\nltk_data')

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w=nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w,intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [10]:
print(words[3:5])
print(classes[3:10])

['Hi', 'there']
['thanks', 'options', 'jokes', 'Identity', 'datetime', 'whatsup', 'haha']


In [11]:
#lemmatize
nltk.download('wordnet')

words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore]
words=sorted(list(set(words)))
classes=sorted(list(set(classes)))

#pickle dump
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


[nltk_data] Downloading package wordnet to C:\Users\WORK
[nltk_data]     PLUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
print("word :")
print(len(words), "unique", words)

word :
129 unique ["'m", '10', '19', 'a', 'age', 'am', 'anyone', 'are', 'ask', 'awesome', 'bad', 'bbye', 'be', 'best', 'bye', 'can', 'contact', 'could', 'covid', 'creator', 'cricket', 'current', 'date', 'day', 'designed', 'developer', 'do', 'doing', 'dumb', 'fine', 'for', 'funny', 'get', 'good', 'goodbye', 'google', 'great', 'haha', 'halo', 'he', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'hot', 'how', 'i', 'idiot', 'india', 'inspiration', 'inspires', 'internet', 'is', 'it', 'joke', 'karan', 'know', 'later', 'latest', 'laugh', 'lmao', 'lol', 'lost', 'made', 'make', 'malik', 'match', 'me', 'motivates', 'namaste', 'news', 'next', 'nice', 'no', 'nope', 'offered', 'ok', 'old', 'programmed', 'programmer', 'provide', 'question', 'riddle', 'rofl', 'score', 'search', 'see', 'set', 'shut', 'song', 'suggest', 'suggestion', 'sup', 'support', 'talk', 'talking', 'tell', 'temperature', 'ten', 'thank', 'thanks', 'that', 'thats', 'the', 'there', 'till', 'time', 'timer', 'to', 'today',

In [13]:
training=[]
output_empty=[0]*len(classes)

for doc in documents:
    bag=[]
    pattern=doc[0]
    pattern=[ lemmatizer.lemmatize(word.lower()) for word in pattern ]
    
    for word in words:
        if word in pattern:
            bag.append(1)
        else:
            bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    
    training.append([bag,output_row])
    
random.shuffle(training)
training=np.array(training, dtype=object)  
X_train=list(training[:,0])
y_train=list(training[:,1]) 

In [14]:
#Model
model=Sequential()
model.add(Dense(128,activation='relu',input_shape=(len(X_train[0]),)))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]),activation='softmax'))

c:\Users\WORK PLUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
#learning rate
SGD= SGD(learning_rate=0.001, decay=1e-6, momentum=0.9, nesterov=True)
adam=keras.optimizers.Adam(0.001)


model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
#model.fit(np.array(X_train),np.array(y_train),epochs=200,batch_size=10,verbose=1)
weights=model.fit(np.array(X_train),np.array(y_train),epochs=200,batch_size=10,verbose=1)    
model.save('mymodel.h5',weights)

Epoch 1/200


c:\Users\WORK PLUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0337 - loss: 3.4024      
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 973us/step - accuracy: 0.0528 - loss: 3.3856   
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 821us/step - accuracy: 0.0586 - loss: 3.3648
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0363 - loss: 3.3637     
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - accuracy: 0.0938 - loss: 3.3123
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step - accuracy: 0.1137 - loss: 3.3211
Epoch 7/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step - accuracy: 0.1121 - loss: 3.2684
Epoch 8/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 978us/step - accuracy: 0.0909 - loss: 3.2449
Epoch 9/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.0947 - loss: 3.1812   
Epoch 10/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 935us/step - accuracy: 0.1425 - loss: 3.1382   
Epoch 11/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - accuracy: 0.0975 - loss: 3.0884   
Epoch 12/200
12/12 ━━━━━━━━━━━━━━━

In [16]:

def clean_up(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[ lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def create_bow(sentence,words):
    sentence_words=clean_up(sentence)
    bag=list(np.zeros(len(words)))
    
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence, model):
    p = create_bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    print(f"Model Prediction: {res}")  # For debugging
    threshold = 0.5
    results = [[i, r] for i, r in enumerate(res) if r > threshold]
    results.sort(key=lambda x: x[1], reverse=True)
    
    return_list = []
    for result in results:
        return_list.append({'intent': classes[result[0]], 'prob': str(result[1])})
    
    return return_list




def getResponse(ints, intents_json):
    if not ints:  # Jika `ints` kosong, kembalikan respons default
        return "Maaf, saya tidak bisa menemukan jawaban untuk itu."

    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    result = "Maaf, saya tidak bisa menemukan jawaban untuk itu."  # Nilai default

    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break  # Hentikan loop setelah menemuthakan tag yang cocok

    return result

    
def chatbot_response(text):
    ints = predict_class(text, model)
    if not ints:  # Tambahkan cek apakah `ints` kosong
        return "Maaf, saya tidak bisa menemukan jawaban untuk itu."
    
    res = getResponse(ints, intents)
    return res
 


def start():
    print("halo this is your bot")
    end = "exit"  # Atur kata untuk keluar dari loop
    while True:
        inp = str(input("You: ")).lower()  # Tambahkan prompt agar pengguna tahu bahwa input diperlukan
        if inp == end:
            break
        if inp == '':
            print("kosong")
        else:
            print(f"bot: {chatbot_response(inp)}" + '\n')


In [17]:
start()

halo this is your bot


In [18]:
print('Help me Learn?')
tag=input('Please enter general category of your question  ')
flag=-1
for i in range(len(intents['intents'])):
    if tag.lower() in intents['intents'][i]['tag']:
        intents['intents'][i]['patterns'].append(input('Enter your message: '))
        intents['intents'][i]['responses'].append(input('Enter expected reply: '))        
        flag=1

if flag==-1:
    
    intents['intents'].append (
        {'tag':tag,
         'patterns': [input('Please enter your message')],
         'responses': [input('Enter expected reply')]})
    
with open('intents.json','w') as outfile:
    outfile.write(json.dumps(intents,indent=4))

Help me Learn?


user input, predict class and get response